In [ ]:
import numpy as np


class Topsis:

    '''
	Create an evaluation matrix consisting of m alternatives and n criteria,
	with the intersection of each alternative and criteria given as {\displaystyle x_{ij}}x_{ij},
	we therefore have a matrix {\displaystyle (x_{ij})_{m\times n}}(x_{{ij}})_{{m\times n}}.
	'''

    def __init__(self, evaluation_matrix, weight_matrix, criteria, normalization=True):
        self.normalization = normalization
        
        # M×N matrix
        self.evaluation_matrix = np.array(evaluation_matrix, dtype="float")
        self.normalized_decision = np.array([])  # 初始化归一矩阵
        self.weighted_normalized = np.array([])  # Weight matrix
        
        # M alternatives (options)
        self.row_size = len(self.evaluation_matrix)

        # N attributes/criteria
        self.column_size = len(self.evaluation_matrix[0])

        # N size weight matrix
        if weight_matrix[0]:
            self.weight_matrix = np.array(weight_matrix, dtype="float")
        self.criteria = np.array(criteria, dtype="float")

    '''
	# Step 2
	The matrix {\displaystyle (x_{ij})_{m\times n}}(x_{{ij}})_{{m\times n}} is then normalised to form the matrix
	'''

    def step_2(self):
        # normalized scores
        self.normalized_decision = self.evaluation_matrix
        if self.normalization:
            sqrd_sum = np.zeros(self.column_size)
            for i in range(self.row_size):
                for j in range(self.column_size):
                    sqrd_sum[j] += self.evaluation_matrix[i, j]**2
            for i in range(self.row_size):
                for j in range(self.column_size):
                    self.normalized_decision[i,
                                             j] = self.evaluation_matrix[i, j]/(sqrd_sum[j]**0.5)
                    

    '''
	# Step 3
	Calculate the weighted normalised decision matrix
	'''

    def step_3(self):
        self.weighted_normalized = self.normalized_decision
        if self.weight_matrix[0]:
            for i in range(self.row_size):
                for j in range(self.column_size):
                    self.weighted_normalized[i, j] *= self.weight_matrix[j]
            
            

    '''
	# Step 4
	Determine the worst alternative {\displaystyle (A_{w})}(A_{w}) and the best alternative {\displaystyle (A_{b})}(A_{b}):
	'''

    def step_4(self):
        self.worst_alternatives = np.zeros(self.column_size)
        self.best_alternatives = np.zeros(self.column_size)
        for i in range(self.column_size):
            if self.criteria[i]:
                self.best_alternatives[i] = max(self.weighted_normalized[:, i])
                self.worst_alternatives[i] = min(self.weighted_normalized[:, i])
            else:
                self.best_alternatives[i] = min(self.weighted_normalized[:, i])
                self.worst_alternatives[i] = max(self.weighted_normalized[:, i])

    '''
	# Step 5
	Calculate the L2-distance between the target alternative {\displaystyle i}i and the worst condition {\displaystyle A_{w}}A_{w}
	{\displaystyle d_{iw}={\sqrt {\sum _{j=1}^{n}(t_{ij}-t_{wj})^{2}}},\quad i=1,2,\ldots ,m,}
	and the distance between the alternative {\displaystyle i}i and the best condition {\displaystyle A_{b}}A_b
	{\displaystyle d_{ib}={\sqrt {\sum _{j=1}^{n}(t_{ij}-t_{bj})^{2}}},\quad i=1,2,\ldots ,m}
	where {\displaystyle d_{iw}}d_{{iw}} and {\displaystyle d_{ib}}d_{{ib}} are L2-norm distances 
	from the target alternative {\displaystyle i}i to the worst and best conditions, respectively.
	'''

    def step_5(self):
        self.worst_distance = np.zeros(self.row_size)
        self.best_distance = np.zeros(self.row_size)

        self.worst_distance_mat = np.copy(self.weighted_normalized) 
        self.best_distance_mat = np.copy(self.weighted_normalized)
        '''
        copy是复制一个独立的副本给另一个变量，如果去除copy相当于两个不同变量名称的变量储存在同一个内存位置。
        所以，改变任意一个变量另一个变量也会跟着改变，因为他们俩的物理内存是同一个
        '''

        for i in range(self.row_size):
            for j in range(self.column_size):
                # 通过self定义的属性在整个实例生命周期都可以使用不必要必须定义在__init__中
                self.worst_distance_mat[i][j] = (self.weighted_normalized[i][j]-self.worst_alternatives[j])**2
                self.best_distance_mat[i][j] = (self.weighted_normalized[i][j]-self.best_alternatives[j])**2
                
                self.worst_distance[i] += self.worst_distance_mat[i][j]
                self.best_distance[i] += self.best_distance_mat[i][j] 

        for i in range(self.row_size):
            self.worst_distance[i] = self.worst_distance[i]**0.5
            self.best_distance[i] = self.best_distance[i]**0.5

    '''
	# Step 6
	Calculate the similarity
	'''

    def step_6(self):
        self.best_similarity = np.zeros(self.row_size)

        for i in range(self.row_size):

            # calculate the similarity to the best condition
            self.best_similarity[i] = self.worst_distance[i] / \
                (self.worst_distance[i]+self.best_distance[i])
            

    def topsis_centrality(self):
        self.step_2()
        self.step_3()
        self.step_4()
        self.step_5()
        self.step_6()
        best_similarity_result = {index: value for index, value in enumerate(self.best_similarity)}
        
    
        return best_similarity_result  # 返回每个节点与最理想解的距离